In [1]:
import pandas as pd
import re
import numpy as np
import glob
import os
from collections import defaultdict
import networkx
from networkx.algorithms.components.connected import connected_components
from packages import npomix
import time
from datetime import datetime
from sklearn.metrics import jaccard_score

In [2]:
start = time.time()

In [3]:
edges_path = "./inputs/gnps_function/ProteoSAFe-METABOLOMICS-SNETS-V2-67f55c7d-download_cytoscape_data/bfcbd4d8e55c43e99283bb029c4bad89..selfloop"
nodes_path = "./inputs/gnps_function/ProteoSAFe-METABOLOMICS-SNETS-V2-67f55c7d-download_cytoscape_data/c985e9c2abe94283828bd01c94645a29.clustersummary"

In [4]:
nodes_df = pd.read_csv(nodes_path,sep='\t')

nodes_df[:5]

,AllGroups,DefaultGroups,EvenOdd,G1,G2,G3,G4,G5,G6,GNPSLinkout_Cluster,...,SpectrumID,UniqueFileSources,UniqueFileSourcesCount,cluster index,componentindex,number of spectra,parent mass,precursor charge,precursor mass,sum(precursor intensity)
0,,G1,1,8,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,Bact_vulg_CL09T03C04_V1.3.mzXML|Bact_sp_9_1_42...,7,2,-1,8,84.043,0,84.043,12004.0
1,,G1,1,14,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,Bacteroides_sp_1_1_30_V1.3.mzXML|Clos_bact_OBR...,12,4,-1,14,84.043,0,84.043,23389.0
2,,G1,1,20,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,Bacteroides_sp_1_1_30_V1.3.mzXML|Bact_sp_1_1_6...,12,6,1846,20,84.044,0,84.044,46212.0
3,,G1,1,16,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,Bacteroides_sp_1_1_30_V1.3.mzXML|Bact_sp_1_1_6...,13,11,1846,16,84.043,0,84.043,37704.0
4,,G1,1,6,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,Bact_thet_CL09T03C10_V1.3.mzXML|Bacteroides_sp...,4,35,-1,6,84.043,0,84.043,12097.0


In [5]:
clusterindex_list = []

for i,r in nodes_df.iterrows():
    if type(r['LibraryID']) != float:
#     if r['LibraryID'] in ['Daidzein','Simvastatin backbone ion']:
        clusterindex_list.append(r['cluster index'])
        
clusterindex_list

[7218,
 7270,
 11268,
 11274,
 11297,
 12849,
 12850,
 12859,
 13179,
 13291,
 13392,
 14846,
 15247,
 15279,
 15546,
 17645,
 17957,
 17958,
 18577,
 18767,
 21372,
 25673,
 25697,
 25796,
 26090,
 26108,
 26241,
 26366,
 26376,
 26385,
 27404,
 27728,
 27734,
 27742,
 27772,
 27873,
 27900,
 28240,
 28245,
 28270,
 28282,
 28310,
 28368,
 28378,
 28380,
 28383,
 28385,
 28514,
 28591,
 29445,
 29551,
 29554,
 29648,
 31135,
 31183,
 31563,
 31605,
 31607,
 31632,
 31671,
 32406,
 32451,
 32575,
 32766,
 33492,
 33618,
 33623,
 33705,
 34282,
 34345,
 34408,
 34861,
 38623,
 38660,
 38666,
 38668,
 39057,
 39061,
 40059,
 40074,
 40119,
 40303,
 40516,
 41280,
 42729,
 45221,
 45224,
 45235,
 46523,
 47104,
 48284,
 48602,
 48603,
 48610,
 48767,
 51668,
 51696,
 51708,
 52756,
 53134,
 53147,
 53254,
 53277,
 53307,
 53613,
 53654,
 53658,
 53693,
 53702,
 53743,
 53762,
 53773,
 53785,
 53947,
 54115,
 54340,
 54474,
 54835,
 55519,
 56883,
 56886,
 56983,
 57118,
 60657,
 60739,
 6

In [6]:
edges_df = pd.read_csv(edges_path,sep='\t')

def get_neighbors(target,dataframe,column1,column2):
    subset1 = dataframe[(dataframe[column1]==target)]
    subcat = subset1.append(dataframe[(dataframe[column2]==target)])
    temp_list = []
    for index,row in subcat.iterrows():
        temp_list.append(subcat[column1][index])
        temp_list.append(subcat[column2][index])
    temp_list = list(np.unique(temp_list))
    return temp_list

def to_edges(l):
    it = iter(l)
    last = next(it)
    for current in it:
        yield last, current
        last = current

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def get_family_dict(components_list,dataframe,dictionary,column1,column2,column3):
    count = 0
    for family in list(components_list):
        count += 1
        for fam_member in family:
            dictionary['MF%s'%count].append(fam_member)
    return dictionary

def main_get_families(gnps_df):
    targets_list = np.unique([gnps_df.CLUSTERID1,gnps_df.CLUSTERID2])
    neighbors_list = []
    for target in targets_list:
        neighbors_list.append(get_neighbors(target,gnps_df,'CLUSTERID1','CLUSTERID2'))
    G = to_graph(neighbors_list)
    C = connected_components(G)
    mf_dict = defaultdict(list)
    mf_dict = get_family_dict(C,gnps_df,mf_dict,'CLUSTERID1','CLUSTERID2','Cosine')
    return mf_dict

mf_dict = main_get_families(edges_df)

mf_dict

defaultdict(list,
            {'MF1': [2],
             'MF2': [4],
             'MF3': [6434,
              4068,
              13445,
              6,
              61689,
              5640,
              2060188,
              11,
              61648,
              3890,
              82035,
              17780,
              17788,
              5625,
              3868],
             'MF4': [35],
             'MF5': [734],
             'MF6': [741],
             'MF7': [742],
             'MF8': [3867, 745, 3355],
             'MF9': [1149],
             'MF10': [14875, 3315],
             'MF11': [3343],
             'MF12': [5181, 3397],
             'MF13': [3695],
             'MF14': [3704],
             'MF15': [7411, 3812, 7412, 7407],
             'MF16': [3988],
             'MF17': [4609],
             'MF18': [15552, 15585, 5012, 25748, 15546],
             'MF19': [5161],
             'MF20': [5297, 5197, 5183],
             'MF21': [5224],
             'MF22': [5303]

In [7]:
for ion in clusterindex_list:
    print(ion)

7218
7270
11268
11274
11297
12849
12850
12859
13179
13291
13392
14846
15247
15279
15546
17645
17957
17958
18577
18767
21372
25673
25697
25796
26090
26108
26241
26366
26376
26385
27404
27728
27734
27742
27772
27873
27900
28240
28245
28270
28282
28310
28368
28378
28380
28383
28385
28514
28591
29445
29551
29554
29648
31135
31183
31563
31605
31607
31632
31671
32406
32451
32575
32766
33492
33618
33623
33705
34282
34345
34408
34861
38623
38660
38666
38668
39057
39061
40059
40074
40119
40303
40516
41280
42729
45221
45224
45235
46523
47104
48284
48602
48603
48610
48767
51668
51696
51708
52756
53134
53147
53254
53277
53307
53613
53654
53658
53693
53702
53743
53762
53773
53785
53947
54115
54340
54474
54835
55519
56883
56886
56983
57118
60657
60739
61645
61724
65814
65834
66538
66554
66825
68060
68110
69849
69851
69875
69887
70419
76907
76909
76912
76926
76929
76930
77108
77217
77345
77401
77526
77766
77801
77806
77820
77827
77831
77843
77845
77846
77852
77854
77861
77863
77872
77881
77890
77895


1079769
1079780
1080113
1087225
1087469
1099736
1100406
1100427
1100463
1101211
1101748
1102828
1103580
1104314
1106102
1106324
1107148
1107295
1108107
1108124
1108125
1108136
1108158
1108174
1108193
1108221
1108236
1108254
1111212
1111695
1111737
1112906
1118213
1118230
1118231
1118247
1118250
1120274
1121939
1121948
1121955
1121958
1121960
1121961
1122006
1122007
1122028
1122046
1122049
1122052
1122090
1122107
1122124
1122126
1122130
1122285
1122296
1123906
1124243
1124360
1124801
1124826
1124848
1124852
1124853
1124864
1124879
1124883
1124889
1124914
1124933
1124935
1124940
1124953
1124974
1124977
1124980
1124990
1125007
1125017
1125031
1125053
1125096
1125138
1125175
1125184
1125198
1125205
1125250
1125266
1125327
1125372
1133764
1134092
1134169
1134191
1134225
1134237
1134264
1134363
1134421
1134781
1134805
1134876
1134934
1135126
1135201
1135221
1135262
1135725
1135808
1135885
1135921
1136189
1136293
1136301
1136510
1140483
1140485
1140759
1141287
1141350
1141356
1141365
1141427


2513072
2513142
2513300
2513825
2514352
2515646
2516965
2519552
2520299
2520300
2523542
2524859
2526173
2527341
2528023
2528825
2529759
2529819
2529822
2529829
2529973
2531361
2532882
2532971
2532992
2533185
2533272
2533485
2533486
2533668
2533710
2534854
2537261
2537340
2537381
2537762
2538339
2539415
2541191
2547172
2547764
2549134
2549135
2549140
2549641
2549767
2549769
2549793
2549797
2550165
2550400
2552727
2554337
2556331
2556423
2556728
2556729
2557055
2558324
2559668
2559777
2559779
2559785
2559798
2559810
2560155
2560173
2561950
2565817
2566949
2566950
2567516
2568010
2568013
2569158
2569212
2569217
2569298
2569745
2569804
2570715
2571139
2572345
2573270
2592191


In [8]:
lcms_file_list = []

for item in nodes_df['UniqueFileSources']:
    for lcms_file in item.split('|'):
        if lcms_file not in lcms_file_list:
            print(lcms_file)
            lcms_file_list.append(lcms_file)

Bact_vulg_CL09T03C04_V1.3.mzXML
Bact_sp_9_1_42FAA_V2.3.mzXML
Blongum44Bv1.3.mzXML
SspCM7v1.2.mzXML
Clos_clos_2_1_49FAA_V1.3.mzXML
SspSR1v1.2.mzXML
Clos_orbi_1_3_50AFAA_V1.3.mzXML
Bacteroides_sp_1_1_30_V1.3.mzXML
Clos_bact_OBRC5-5_V1.2.mzXML
SspOBRC6v1.2.mzXML
Bact_frag_CL07T12C05_V1.3.mzXML
Bact_cacc_CL03T12C61_V1.3.mzXML
SspCM6v1.2.mzXML
Para_merd_CL03T12C32_V1.3.mzXML
Bact_dore_CL02T00C15_V1.3.mzXML
Bact_sp_1_1_6_V2.3.mzXML
Bact_cell_CL02T12C19_V1.3.mzXML
Bact_frag_CL07T00C01_V1.3.mzXML
Bact_ster_CC31F_V1.3.mzXML
Acti_vis_C505_V3.3.mzXML
Bact_thet_CL09T03C10_V1.3.mzXML
GCA_000012265.2.mzXML
GCA_000012265.1.mzXML
GCA_000506385.2.mzXML
GCA_003324555.1.mzXML
GCA_001562525.1.mzXML
GCA_001562525.2.mzXML
GCA_000506385.4.mzXML
GCA_003324555.2.mzXML
Blongum35Bv1.3.mzXML
Bact_dore_CL02T12C06_V1.3.mzXML
Bifi_brev_HPH0326_V1.3.mzXML
SspBS29av1.2.mzXML
Bact_frag_CL03T12C07_V1.3.mzXML
ERX2291258.1.mzXML
ERX2291921.1.mzXML
ERX2291789.1.mzXML
BspMSTE12v1.3.mzXML
B111.4.mzML
7961.3.mzML
M92.1.mzML
M

GCA_010672205.5.mzXML
GCA_010672385.4.mzXML
GCA_010672305.5.mzXML
GCA_010672385.5.mzXML
GCA_010672205.4.mzXML
GCA_010672365.7.mzXML
GCA_010672205.2.mzXML
GCA_010672205.9.mzXML
GCA_010672455.1.mzXML
GCA_010672365.9.mzXML
GCA_010672385.7.mzXML
GCA_010672695.1.mzXML
GCA_010692685.5.mzXML
GCA_010672695.5.mzXML
GCA_010692685.8.mzXML
GCA_010672305.2.mzXML
GCA_010672385.8.mzXML
GCA_010672205.6.mzXML
GCA_010672205.7.mzXML
GCA_000710405.1.mzXML
2747842505.1.mzXML
2518285564.1.mzXML
SspACS2v1.2.mzXML
ERX2291818.1.mzXML
ERS4341483.1.mzXML
ERS4341631.1.mzXML
ERS4346463.1.mzXML
ERS4341603.1.mzXML
ERS4346533.1.mzXML
ERS4346513.1.mzXML
ERS4341378.1.mzXML
ERS4341608.1.mzXML
ERS4341644.1.mzXML
ERS4341372.1.mzXML
ERS4341451.1.mzXML
ERS4341580.1.mzXML
ERS4341428.1.mzXML
ERS4341647.1.mzXML
ERS4346438.1.mzXML
ERS4346448.1.mzXML
ERS4341424.1.mzXML
ERS4346439.1.mzXML
ERS4341512.1.mzXML
ERS4341577.1.mzXML
ERS4341711.1.mzXML
ERS4341377.1.mzXML
Kleb_oxyt_10-5243_V1.1.mzXML
Lach_bact_7_1_58FAA_V1.1.mzXML
ERS4341

ERX2291891.1.mzXML
ERS4341417.1.mzXML
ERS4341680.1.mzXML
ERS4346500.1.mzXML
ERS4341582.1.mzXML
ERS4341531.1.mzXML
ERS4346449.1.mzXML
ERS4341368.1.mzXML
ERS4341553.1.mzXML
ERS4341472.1.mzXML
ERS4341633.1.mzXML
ERS4346477.1.mzXML
ERS4341514.1.mzXML
ERS4346486.1.mzXML
ERX2291935.1.mzXML
ERX2291698.1.mzXML
ERS4341538.1.mzXML
ERS4341515.1.mzXML
ERS4341642.1.mzXML
ERS4341476.1.mzXML
ERS4346572.1.mzXML
ERS4346501.1.mzXML
ERS4346434.1.mzXML
ERS4346418.1.mzXML
ERS4341606.1.mzXML
ERS4341485.1.mzXML
ERS4346441.1.mzXML
ERS4341693.1.mzXML
ERS4341670.1.mzXML
ERS4341636.1.mzXML
ERS4341375.1.mzXML
ERS4346406.1.mzXML
ERS4341509.1.mzXML
ERS4346456.1.mzXML
ERS4341690.1.mzXML
ERS4341482.1.mzXML
ERS4346582.1.mzXML
ERS4341576.1.mzXML
ERX2291405.1.mzXML
ERS4341573.1.mzXML
ERS4341443.1.mzXML
ERX2291923.1.mzXML
ERS4341392.1.mzXML
ERS4341520.1.mzXML
ERS4341493.1.mzXML
ERS4341617.1.mzXML
ERX2291784.1.mzXML
ERS4356252.1.mzXML
ERS4356215.1.mzXML
ERS4356315.1.mzXML
ERS4356279.1.mzXML
ERS4356178.1.mzXML
ERS4356247.1

GCA_009712075.2.mzML
GCA_009712075.1.mzML
GCA_009711935.2.mzML
GCA_009711935.1.mzML
GCA_010672485.1.mzXML
GCA_010672535.1.mzXML
GCA_010672755.1.mzXML
GCA_010672795.1.mzXML
GCA_002994615.1.mzXML
GCA_002994635.1.mzXML
GCA_002994615.6.mzXML
GCA_000737335.6.mzXML
GCA_008121535.2.mzML
GCA_008121535.1.mzML
GCA_000317515.1.mzML
GCF_000719925.1.mzML
GCA_000734895.2.mzML
GCA_000734895.1.mzML
GCF_000719925.2.mzML
GCA_000170895.3.mzXML
GCA_000737335.5.mzXML
GCA_010692445.1.mzXML
GCA_000737335.4.mzXML
GCA_002994615.3.mzXML
GCA_002994615.2.mzXML
GCA_002994615.5.mzXML
GCA_000170895.2.mzXML
GCA_000170895.4.mzXML
GCA_000170895.6.mzXML
GCA_000737335.2.mzXML
GCA_002994615.7.mzXML
GCA_002994635.2.mzXML
GCA_000737335.3.mzXML
GCA_000170895.5.mzXML
GCA_002104455.1.mzXML
GCA_002104455.2.mzXML
GCA_000737335.1.mzXML
GCA_000377165.5.mzXML
GCA_000158915.5.mzXML
GCA_000238295.1.mzXML
GCA_000377145.5.mzXML
GCA_000213055.5.mzXML
AXCT02000000.1.mzXML
GCA_000238295.2.mzXML
B1034.6.mzXML
B1034.5.mzXML
GCA_010692535.1.

GCA_000424765.5.mzXML
GCA_000739105.5.mzXML
GCA_000377545.5.mzXML
CENA71.1.mzML
GCA_000377545.4.mzXML
GCA_000702365.5.mzXML
GCA_000377965.24.mzXML
GCA_000377965.15.mzXML
GCA_000158915.6.mzXML
GCA_000377965.10.mzXML
GCA_000240165.10.mzXML
2517434008.2.mzXML
GCA_000240165.11.mzXML
GCA_000240165.14.mzXML
GCA_000739105.6.mzXML
GCA_000240165.17.mzXML
GCA_000377965.21.mzXML
GCA_000240165.20.mzXML
GCA_000377145.4.mzXML
GCA_000377965.18.mzXML
GCA_000158975.4.mzXML
GCA_000240165.44.mzXML
GCA_000158915.4.mzXML
GCA_000240165.19.mzXML
GCA_002994635.4.mzXML
GCA_000240165.18.mzXML
GCA_000240165.21.mzXML
GCA_000240165.16.mzXML
GCA_002994635.3.mzXML
GCA_000377965.12.mzXML
2548876909.1.mzXML
GCA_000515055.4.mzXML
GCA_000739105.4.mzXML
2518285561.2.mzXML
2518285561.1.mzXML
2548876909.3.mzXML
GCA_000377165.3.mzXML
GCA_000203835.8.mzXML
2518285562.18.mzXML
GCA_000377965.17.mzXML
GCA_000424965.6.mzXML
GCA_000377965.20.mzXML
GCA_002994635.5.mzXML
GCA_002994635.6.mzXML
GCA_000156435.5.mzXML
GCA_001942495.1.m

In [9]:
len(lcms_file_list)

3240

In [10]:
def get_presence_files(nodes_df):
    for unique_list in list(nodes_df[nodes_df['cluster index'] == ion]['UniqueFileSources']):
        return list(unique_list.split('|'))

all_rows_list,testing_indexes_list = [],[]
for ion in clusterindex_list:
    subset_edges = edges_df[(edges_df.CLUSTERID1 == ion) | (edges_df.CLUSTERID2 == ion)]
    cosine = round(max(subset_edges['Cosine']),2)
    presence_list = get_presence_files(nodes_df)
    single_row_list = []
    for lcms_file in lcms_file_list:
        if lcms_file in presence_list:
            single_row_list.append(cosine)
        else:
            single_row_list.append(0)
    all_rows_list.append(single_row_list)
    testing_indexes_list.append(ion)
    
all_rows_list

pre_testing_df = pd.DataFrame(all_rows_list,index=testing_indexes_list,columns=lcms_file_list)

pre_testing_df

,Bact_vulg_CL09T03C04_V1.3.mzXML,Bact_sp_9_1_42FAA_V2.3.mzXML,Blongum44Bv1.3.mzXML,SspCM7v1.2.mzXML,Clos_clos_2_1_49FAA_V1.3.mzXML,SspSR1v1.2.mzXML,Clos_orbi_1_3_50AFAA_V1.3.mzXML,Bacteroides_sp_1_1_30_V1.3.mzXML,Clos_bact_OBRC5-5_V1.2.mzXML,SspOBRC6v1.2.mzXML,...,GCA_000087965.7.mzXML,GCA_000087965.5.mzXML,GCA_000087965.3.mzXML,GCA_000087965.2.mzXML,GCA_000087965.8.mzXML,GCA_000087965.9.mzXML,GCA_000087965.6.mzXML,GCA_000087965.1.mzXML,GCA_000377145.7.mzXML,2517434008.3.mzXML
7218,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7270,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
11268,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
11274,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
11297,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570715,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2571139,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2572345,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2573270,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [11]:
def get_final_df(training_df,testing_df,neighbors_array,results_folder):
#     final_df = pd.DataFrame(columns=('metabolite_ID','predicted_GCFs','max_jaccard','parent_mass','peak_count'))
    final_df = pd.DataFrame(columns=('metabolite_ID','predicted_GCFs','max_jaccard'))
    for i,ccms_id in enumerate(testing_df.index):
        jaccard_scores = []
        for j in range(0,len(neighbors_array[i])):
            query_bgc = neighbors_array[i][j]
            bgc_fp = training_df[training_df['label'] == query_bgc].iloc[0]
            bgc_fp = bgc_fp.drop("label")
            ms_fp = testing_df.loc[ccms_id]
            bgc_binary = npomix.get_binary(bgc_fp)
            ms_binary = npomix.get_binary(ms_fp)
            jaccard_scores.append(jaccard_score(bgc_binary,ms_binary))
        max_jaccard = round(max(jaccard_scores),2)
#         pepmass,peak_count = get_ms2_metadata(ccms_id)
        final_df.loc[i] = ccms_id,neighbors_array[i],max_jaccard
#     final_df.to_csv("%sfinal_df-NPOmix1.0-%s.txt"%(results_folder,current_date),sep="\t",index_label=False)
    return final_df

In [12]:
ena_df_file = "./ena_dict-210315.csv"
input_bigscape_net = "./bigscape_all_c030.txt"
antismash_folder = "./inputs/gnps_function/antismash_only_gbk/"
results_folder = "./lastest_results/npomix2results_gnps_validation/"

current_date = datetime.today().strftime('%Y%m%d')

if not os.path.isdir(results_folder):
    os.mkdir(results_folder)

k_value = 3

merged_ispec_mat = npomix.get_merged_ispec_mat(pre_testing_df)
merged_ispec_mat = npomix.renaming_merged_ispec_mat(ena_df_file,merged_ispec_mat)
print('Obtaining BiG-SCAPE dataframe and BiG-SCAPE dictionary')
bigscape_df,bigscape_dict = npomix.get_bigscape_df(ena_df_file,input_bigscape_net)
bigscape_df,bigscape_dict2 = npomix.rename_bigscape_df(antismash_folder,bigscape_df,bigscape_dict)
print('BiG-SCAPE create with %s GCFs'%len(bigscape_dict))
strain_list,bgcs_list = npomix.get_strain_list(bigscape_df)
print('Creating training dataframe')
affinity_df,affinity_bgcs = npomix.get_pre_training_df(bigscape_df,bigscape_dict2,strain_list,bgcs_list)
affinity_df = npomix.renaming_affinity_df(affinity_df)
networked_cols = npomix.get_networked_cols(merged_ispec_mat,affinity_df)
training_df,training_bgcs = npomix.get_training_df(affinity_df,networked_cols,results_folder,affinity_bgcs)
bgcs_df = pd.DataFrame(training_bgcs, columns=['bgcs'])
testing_df = npomix.get_testing_df(merged_ispec_mat,networked_cols,results_folder)
print('Running KNN using k equals to %s'%k_value)
neighbors_array = npomix.running_knn(training_df,testing_df,k_value)
print('Creating final dataframe')
final_df = get_final_df(training_df,testing_df,neighbors_array,results_folder)

Obtaining BiG-SCAPE dataframe and BiG-SCAPE dictionary
BiG-SCAPE create with 997 GCFs
Creating training dataframe
Running KNN using k equals to 3
Creating final dataframe


In [13]:
final_df

,metabolite_ID,predicted_GCFs,max_jaccard
0,7218,"[GCF209, GCF168, GCF134]",0.00
1,7270,"[GCF94, GCF94, GCF94]",0.24
2,11268,"[GCF364, GCF360, GCF364]",0.60
3,11274,"[GCF8, GCF6, GCF6]",0.00
4,11297,"[GCF81, GCF81, GCF81]",0.42
...,...,...,...
4634,2570715,"[GCF445, GCF465, GCF360]",1.00
4635,2571139,"[GCF736, GCF736, GCF736]",0.67
4636,2572345,"[GCF713, GCF740, GCF740]",0.50
4637,2573270,"[GCF741, GCF737, GCF737]",1.00


In [14]:
mibig_df = pd.read_csv("./matched_mibig_gnps_update.tsv",sep='\t')

mibig_name_dict = dict(zip(mibig_df['mibig_id'],mibig_df['mibig_name']))

ccmsid_mibig_dict = dict(zip(mibig_df['# mgf_spectrum_id'],mibig_df['mibig_id']))

ccmsid_mibig_dict

{'CCMSLIB00000001552': 'BGC0001000',
 'CCMSLIB00000006865': 'BGC0000001',
 'CCMSLIB00000075009': 'BGC0000950',
 'CCMSLIB00000075016': 'BGC0000950',
 'CCMSLIB00000075305': 'BGC0000055',
 'CCMSLIB00000075306': 'BGC0000055',
 'CCMSLIB00000075307': 'BGC0000706',
 'CCMSLIB00000075308': 'BGC0000706',
 'CCMSLIB00000075309': 'BGC0000724',
 'CCMSLIB00000075310': 'BGC0000724',
 'CCMSLIB00000075311': 'BGC0000455',
 'CCMSLIB00000075312': 'BGC0000455',
 'CCMSLIB00000075313': 'BGC0001453',
 'CCMSLIB00000075320': 'BGC0000985',
 'CCMSLIB00000075321': 'BGC0000985',
 'CCMSLIB00000075322': 'BGC0000985',
 'CCMSLIB00000075323': 'BGC0000985',
 'CCMSLIB00000075324': 'BGC0000985',
 'CCMSLIB00000075325': 'BGC0000985',
 'CCMSLIB00000075331': 'BGC0000016',
 'CCMSLIB00000075332': 'BGC0000016',
 'CCMSLIB00000077217': 'BGC0001310',
 'CCMSLIB00000077218': 'BGC0001310',
 'CCMSLIB00000078898': 'BGC0000901',
 'CCMSLIB00000081213': 'BGC0000389',
 'CCMSLIB00000081265': 'BGC0000820',
 'CCMSLIB00000081266': 'BGC0000820',
 

In [31]:
for i,r in final_df[final_df['max_jaccard'] > 0].iterrows():
    libhit = str(nodes_df[nodes_df['cluster index'] == r['metabolite_ID']]['LibraryID'].item())
    specID = str(nodes_df[nodes_df['cluster index'] == r['metabolite_ID']]['SpectrumID'].item())
    bgchits_list = []
    match = False
    for gcf in r['predicted_GCFs']:
        for bgc in bigscape_dict2[gcf]:
            bgc = bgc.split('.')[0]
            if 'BGC' in bgc:
                if bgc in mibig_name_dict:
                    bgchits_list.append(mibig_name_dict[bgc])
                    if specID in ccmsid_mibig_dict:
                        match = True
                        if ccmsid_mibig_dict[specID] == bgc:
                            print(i,'true',gcf,specID,ccmsid_mibig_dict[specID],bgc,libhit)
                        else:
                            print(i,'false',gcf,specID,ccmsid_mibig_dict[specID],bgc,'mibig=%s'%mibig_name_dict[bgc],'gnps=%s'%libhit)
#                     else:
#                         print('missing %s'%specID)
#                 else:
#                     print('missing %s'%bgc)
    if match == True:
        print(r['predicted_GCFs'],'\n')

3916 true GCF347 CCMSLIB00000223877 BGC0001830 BGC0001830 Rosamicin
3916 true GCF347 CCMSLIB00000223877 BGC0001830 BGC0001830 Rosamicin
['GCF347' 'GCF347' 'GCF352'] 

4147 false GCF149 CCMSLIB00005435752 BGC0000343 BGC0000664 mibig=isorenieratene gnps=Enterobactin
['GCF494' 'GCF499' 'GCF149'] 

4478 true GCF476 CCMSLIB00000579285 BGC0001088 BGC0001088 Albicidin (LTQ-Orbitrap)
['GCF180' 'GCF476' 'GCF180'] 

4506 true GCF476 CCMSLIB00004681481 BGC0001088 BGC0001088 Beta-Methoxy-Albicidin
4506 true GCF476 CCMSLIB00004681481 BGC0001088 BGC0001088 Beta-Methoxy-Albicidin
['GCF180' 'GCF476' 'GCF476'] 

4507 true GCF476 CCMSLIB00004681481 BGC0001088 BGC0001088 Beta-Methoxy-Albicidin
4507 true GCF476 CCMSLIB00004681481 BGC0001088 BGC0001088 Beta-Methoxy-Albicidin
['GCF476' 'GCF476' 'GCF180'] 

4539 true GCF476 CCMSLIB00004681486 BGC0001088 BGC0001088 Carbamoyl-Beta-Methoxy-Albicidin
4539 true GCF476 CCMSLIB00004681486 BGC0001088 BGC0001088 Carbamoyl-Beta-Methoxy-Albicidin
['GCF476' 'GCF476' 'GC

In [18]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
run_time = "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
print(run_time)

00:08:45.32
